In [25]:
import pandas as pd
from pathlib import Path
import uuid

In [5]:
fincausal_task1_iteration1_train_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_1/train.csv')
semeval_all_data_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/extra/semeval.csv')

In [3]:
semeval_all_data_df = pd.read_csv(semeval_all_data_path)
semeval_all_data_df.head()

,sentence,relation,causal
0,The system as described above has its greatest...,Component-Whole,0
1,The child was carefully wrapped and bound into...,Other,0
2,The author of a keygen uses a disassembler to ...,Instrument-Agency,0
3,A misty ridge uprises from the surge.,Other,0
4,The student association is the voice of the un...,Member-Collection,0


In [21]:
semeval_all_data_df.shape

(10717, 3)

In [6]:
fincausal_task1_iteration1_train_df = pd.read_csv(fincausal_task1_iteration1_train_path)
fincausal_task1_iteration1_train_df.head()

,Index,Text,Gold,unique_id
0,81.00009,Look at which clients are delivering increasi...,0,849c40bc-8fce-433b-8dc8-6af727480679
1,357.00082,From foundation modules that introduce asset ...,0,694a6824-613c-4978-9e17-a45b77e2b485
2,140.00047,Cautionary Note to United States Investors T...,0,c4c3f26d-cae5-4178-a469-ab343c6ea3c9
3,166.00001,Upcoming events Learn about the requirements...,0,48a694c5-9372-48d2-b2cb-df9461c713fe
4,48.00012,"The Midwest, where Buffett is from, has a his...",0,a363e188-0393-433b-a86c-94d268eb473c


In [35]:
fincausal_task1_iteration1_train_df.shape

(15575, 4)

In [23]:
# what's the distribution of label?
causal_distribution = fincausal_task1_iteration1_train_df.Gold.value_counts() / len(fincausal_task1_iteration1_train_df)
causal_distribution

0    0.927897
1    0.072103
Name: Gold, dtype: float64

In [18]:
# how many causal can I take from semeval?
semeval_noncausal = semeval_all_data_df.loc[semeval_all_data_df.causal == 0]
semeval_causal = semeval_all_data_df.loc[semeval_all_data_df.causal == 1]

semeval_causal_resampled = semeval_causal.sample(n=int((causal_distribution[1]*100*len(semeval_noncausal))/(100-(causal_distribution[1]*100))))

semeval_resampled = pd.concat([semeval_noncausal, semeval_causal_resampled])
semeval_resampled.causal.value_counts() / len(semeval_resampled)

0    0.927929
1    0.072071
Name: causal, dtype: float64

In [30]:
# combine both the datasets
semval_resampled_formatted_df = pd.DataFrame({
    'Index': None,
    'Text': semeval_resampled.sentence,
    'Gold': semeval_resampled.causal,
    'unique_id': [str(uuid.uuid4()) for _ in range(len(semeval_resampled))]
})
semval_resampled_formatted_df.head()

,Index,Text,Gold,unique_id
0,None,The system as described above has its greatest...,0,be12b158-d072-40f1-84a3-196bf7835427
1,None,The child was carefully wrapped and bound into...,0,f1f8baf1-a759-4566-a582-ff34670404cb
2,None,The author of a keygen uses a disassembler to ...,0,3c17189a-fe8e-4a68-aded-22891d50a5d2
3,None,A misty ridge uprises from the surge.,0,44a22a95-d7dc-42fe-9363-0f064b1bf5aa
4,None,The student association is the voice of the un...,0,38ef1f8c-9b96-4b2d-a7e6-fdc1bb09cbd0


In [32]:
semval_resampled_formatted_df.Gold.value_counts() / len(semval_resampled_formatted_df)

0    0.927929
1    0.072071
Name: Gold, dtype: float64

In [33]:
fincausal_task1_train_combined_semeval = pd.concat([fincausal_task1_iteration1_train_df, semval_resampled_formatted_df])
fincausal_task1_train_combined_semeval.shape

(25690, 4)

In [34]:
fincausal_task1_train_combined_semeval.Gold.value_counts() / len(fincausal_task1_train_combined_semeval)

0    0.92791
1    0.07209
Name: Gold, dtype: float64

In [36]:
fincausal_task1_train_combined_semeval.to_csv('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_1/train_with_semeval.csv', index=False)